# Adult Income dataset

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.utils.data as Data
import torch.nn.functional as F

import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
x = np.load('data.npy')
y = np.load('labels.npy')

In [3]:
print(x.shape)
print(y.shape)
print(x[3])

(45222, 67)
(45222,)
[35  9  0  0 40  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1
  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
  0  1  0  0  0  0  0  0  0  0  0  1  0  1  1  0  0  0  0]


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    stratify=y, 
                                                    test_size=0.15)

In [5]:
print(x_test.shape)
print(y_test.shape)
len(x_test)

(6784, 67)
(6784,)


6784

In [6]:
class ImgLoader(Dataset):
    def __init__(self, x, y,iscuda=False):
        self.X = np.array(x, dtype = np.float32)
        self.y = np.array(y, dtype = np.float32)
#         self.cuda = iscuda
    
    def __getitem__(self, index):
        x_val = self.X[index]
        x_val = torch.from_numpy(x_val).permute(2, 1, 0)
        y_val = torch.from_numpy(np.array([self.y[index]]))
#         if self.cuda:
#             x_val = x_val.cuda()
#             y_val = y_val.cuda()
        return x_val, y_val

    def __len__(self):
        return len(self.X)

In [7]:
# data_tensor = torch.from_numpy(x_test).float()
# target_tensor = torch.from_numpy(y_test).long()
# test_dataset = Data.TensorDataset(data_tensor, target_tensor)

In [9]:
batch_size = 3

data_tensor = torch.from_numpy(x_train).float()
target_tensor = torch.from_numpy(y_train).long()
train_dataset = Data.TensorDataset(data_tensor, target_tensor)

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

In [10]:
data_tensor = torch.from_numpy(x_test).float()
target_tensor = torch.from_numpy(y_test).long()
test_dataset = Data.TensorDataset(data_tensor, target_tensor)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

In [11]:
input_size = 67
h_size1 = 55
h_size2 = 35
num_epochs = 5
num_classes = 2

In [15]:
class Net(nn.Module):
    def __init__(self, input_size, h_size1, h_size2, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, h_size1),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(h_size1, h_size2),
            nn.ReLU())
        self.fc3 = nn.Sequential(
            nn.Linear(h_size2,num_classes))
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
# dataloaders = dict()
# dataloaders['train'] = train_loader
# dataloaders['val'] = test_loader

# dataset_sizes = {'train': len(x_train), 'val':len(x_train)}
# use_gpu = torch.cuda.is_available()

In [26]:
def eval(model):
    model.train(False)
    count = 0
    for x,y in zip(x_test,y_test):
        x = torch.from_numpy(np.array([x]))#.permute(0, 3, 1, 2)
        x = x.type(torch.FloatTensor)
        if use_cuda:
            x = x.cuda()
        out = model(Variable(x))
#         print(out.shape)
        label = np.argmax(out.data.cpu().numpy()) # needs to be optimized
        if y == label:
            count += 1
    print(count * 1.0 / len(x_test))

In [18]:
epochs = 10
criterion = nn.CrossEntropyLoss()
net = Net(input_size, h_size1, h_size2, num_classes)
use_cuda = torch.cuda.is_available()
if use_cuda:
    net = net.cuda()

In [19]:
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr= 0.001)

In [27]:
# optimizer = optim.SGD(params=net.parameters(), lr=0.003)

for epoch in range(epochs):
    losses = []
    net.train(True)
    for i ,(x, y) in enumerate(train_loader):
        if use_cuda:
            x = x.cuda()
            y = y.cuda()
        optimizer.zero_grad()

        inputs =  Variable(x)
        output = net(inputs)
        targets = Variable(y.squeeze(-1))
        targets = targets.type(torch.LongTensor)

        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.data[0])
#         break
#     break
    if (epoch+1) % 1 == 0:
        print('Epoch %d Loss %.4f  ' % (epoch+1, np.average(losses)))
        print('test accuracy')
        eval(net)

/home/yash/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 1 Loss 0.3469  
test accuracy
0.8378537735849056
Epoch 2 Loss 0.3425  
test accuracy
0.8421285377358491
Epoch 3 Loss 0.3372  
test accuracy
0.8391804245283019
Epoch 4 Loss 0.3353  
test accuracy
0.8427181603773585
Epoch 5 Loss 0.3327  
test accuracy
0.8433077830188679
Epoch 6 Loss 0.3316  
test accuracy
0.8310731132075472
Epoch 7 Loss 0.3320  
test accuracy
0.8343160377358491
Epoch 8 Loss 0.3290  
test accuracy
0.8375589622641509
Epoch 9 Loss 0.3305  
test accuracy
0.8461084905660378
Epoch 10 Loss 0.3303  
test accuracy
0.8465507075471698


In [24]:
y.squeeze(-1)


tensor([0, 1, 1])

In [ ]:

# def train_model(model, criterion,optimizer, num_epochs):
#     f = open("Iterations.txt", "w+")
#     best_model_wts = model.state_dict()
#     best_val_acc = 0.0
#     best_train_acc = 0.0
#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs-1))
#         print('-' * 10)
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train(True)
#             else:
#                 model.train(False)
#         running_loss = 0.0
#         running_corrects = 0.0
#         for data in dataloaders[phase]:
#             inputs, label = data
#             if use_gpu:
#                 inputs = Variable(input.cuda())
#                 labels = Variable(labels.cuda())
#             else:
#                 inputs, labels = Variable(inputs), Variable(label)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             _, preds = torch.max(outputs.data, 1)
#             loss = criterion(outputs, labels)
            
#             if phase == 'train':
#                 loss.backward()
#                 optimizer.step()
#             running_loss += loss.data[0]
#             running_corrects += torch.sum(preds == label)
#             epoch_loss = running_loss/ dataset_sizes[phase]
#             epoch_acc = running_corrects / dataset_sizes[phase]

#         print('{} Loss: {:.8f} Accuracy: {:.4f}'.format(phase, epoch_loss, epoch_acc))
#         f.write('{} Loss: {:.8f} Accuracy: {:.4f}\n'.format(phase, epoch_loss, epoch_acc))    
            
#         if phase == 'val' and epoch_acc > best_val_acc:
#             best_val_acc = epoch_acc
#             best_model_wts = model.state_dict()
#         if phase == 'train' and epoch_acc > best_train_acc:
#             best_train_acc = epoch_acc
#             best_model_wts = model.state_dict()
#     f.close()
#     print('Best Val Acc: {:4f}'.format(best_val_acc))
#     model.load_state_dict(best_model_wts)
#     return model, best_train_acc, best_val_acc

# if use_gpu:
#     model_ft, train_acc, test_acc = train_model(net.cuda(), criterion, optimizer, num_epochs)
# else:
#     model_ft, train_acc, test_acc = train_model(net, criterion, optimizer, num_epochs)